In [1]:
#importing libraries
import requests
import pandas as pd
import json
from datetime import date, datetime, timedelta
import os
import mysql.connector as msql
from mysql.connector import Error


In [2]:
#Fetching data from USGS API, with necessary parameters for the year 2017

start_date = date(2017, 1, 1)
end_date = date(2018, 1, 1)

values=[]

while end_date>start_date:
       
    end_time=start_date+timedelta(days=5)
    base_url="https://earthquake.usgs.gov/fdsnws/event/1/query?"
    paramss = {"format": "geojson", "starttime":start_date , "endtime":end_time  }
    file=requests.get(base_url, params = paramss)
    data = json.loads(file.text)
    start_date=end_time


    for i in data["features"]:
        values.append(i["properties"])

data=pd.DataFrame(values)
            

In [3]:
data.head()

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,ids,sources,types,nst,dmin,rms,gap,magType,type,title
0,1.40,"56 km SSW of Cantwell, Alaska",1483660672516,1560812262490,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ak14976205,ak0178nq18z,",",ak,ak,",",associate,origin,phase-data,",NaN,NaN,0.4300,NaN,ml,earthquake,"M 1.4 - 56 km SSW of Cantwell, Alaska"
1,1.25,"3km SE of Frazier Park, CA",1483660403630,1486588764323,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ci37781024,",",ci,",",nearby-cities,origin,phase-data,scitech-link,",19.0,0.04354,0.1900,83.00,ml,earthquake,"M 1.3 - 3km SE of Frazier Park, CA"
2,0.70,"27 km WSW of Hawthorne, Nevada",1483660265260,1530555574174,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",nn00581801,",",nn,",",origin,phase-data,",7.0,0.04100,0.0600,98.28,ml,earthquake,"M 0.7 - 27 km WSW of Hawthorne, Nevada"
3,2.30,"22 km NW of Portage Creek, Alaska",1483660241918,1560812261958,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ak14976203,ak0178noif3,",",ak,ak,",",associate,origin,phase-data,",NaN,NaN,0.5000,NaN,ml,earthquake,"M 2.3 - 22 km NW of Portage Creek, Alaska"
4,1.00,"27 km WSW of Hawthorne, Nevada",1483660042931,1530555574068,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",nn00572605,",",nn,",",origin,phase-data,",8.0,0.04300,0.1549,101.05,ml,earthquake,"M 1.0 - 27 km WSW of Hawthorne, Nevada"


In [4]:
#converting the Unix timestamp to datetime
occurence_date=[]
occurence_time=[]
for i in range(len(data)):
    occurence_date.append(datetime.fromtimestamp(data.time[i]/1000).strftime("%Y-%m-%d %H:%M:%S"))
    occurence_time.append(datetime.fromtimestamp(data.time[i]/1000).strftime("%H"))

data['time']=occurence_date
data['occurence(hour)']=occurence_time

In [5]:
#Converting magnitude and hour of day fields into bins
bins=[0,1,2,3,4,5,6,10]
label=['0-1','1-2','2-3','3-4','4-5','5-6','6+']
data['mag_bins'] = pd.cut(data['mag'], bins=bins, labels=label)
data['mag_bins']

0         1-2
1         1-2
2         0-1
3         2-3
4         0-1
         ... 
134654    1-2
134655    1-2
134656    1-2
134657    NaN
134658    0-1
Name: mag_bins, Length: 134659, dtype: category
Categories (7, object): ['0-1' < '1-2' < '2-3' < '3-4' < '4-5' < '5-6' < '6+']

In [6]:
#Following standard nomenclature for columns
data=data.rename(columns={"type":"product_type", "types":"occurence_types", "status":"review_status", "event_ids":"ids","event_code":"code"})
data.head()

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,occurence_types,nst,dmin,rms,gap,magType,product_type,title,occurence(hour),mag_bins
0,1.40,"56 km SSW of Cantwell, Alaska",2017-01-05 18:57:52,1560812262490,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",associate,origin,phase-data,",NaN,NaN,0.4300,NaN,ml,earthquake,"M 1.4 - 56 km SSW of Cantwell, Alaska",18,1-2
1,1.25,"3km SE of Frazier Park, CA",2017-01-05 18:53:23,1486588764323,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",nearby-cities,origin,phase-data,scitech-link,",19.0,0.04354,0.1900,83.00,ml,earthquake,"M 1.3 - 3km SE of Frazier Park, CA",18,1-2
2,0.70,"27 km WSW of Hawthorne, Nevada",2017-01-05 18:51:05,1530555574174,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",origin,phase-data,",7.0,0.04100,0.0600,98.28,ml,earthquake,"M 0.7 - 27 km WSW of Hawthorne, Nevada",18,0-1
3,2.30,"22 km NW of Portage Creek, Alaska",2017-01-05 18:50:41,1560812261958,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",associate,origin,phase-data,",NaN,NaN,0.5000,NaN,ml,earthquake,"M 2.3 - 22 km NW of Portage Creek, Alaska",18,2-3
4,1.00,"27 km WSW of Hawthorne, Nevada",2017-01-05 18:47:22,1530555574068,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",origin,phase-data,",8.0,0.04300,0.1549,101.05,ml,earthquake,"M 1.0 - 27 km WSW of Hawthorne, Nevada",18,0-1


In [7]:
data.columns

Index(['mag', 'place', 'time', 'updated', 'tz', 'url', 'detail', 'felt', 'cdi',
       'mmi', 'alert', 'review_status', 'tsunami', 'sig', 'net', 'code', 'ids',
       'sources', 'occurence_types', 'nst', 'dmin', 'rms', 'gap', 'magType',
       'product_type', 'title', 'occurence(hour)', 'mag_bins'],
      dtype='object')

#### Data Dictonary

#Below is the list of all the entities that fetched from the USGS' APIs.

** alert **
  - Data Type : String
  - Typical Values : “green”, “yellow”, “orange”, “red”.
  - Description :The alert level from the PAGER earthquake impact scale.

** cdi **
  - Data Type : Decimal
  - Typical Values : [0.0, 10.0]
  - Description :The maximum reported intensity for the event. Computed by DYFI. While typically reported as a roman numeral, for the purposes of this API, intensity is expected as the decimal equivalent of the roman numeral. Learn more about magnitude vs. intensity.


** code **
  - Data Type : String
  - Typical Values : "2013lgaz", "c000f1jy", "71935551"
  - Description :An identifying code assigned by - and unique from - the corresponding source for the event.


** detail **
  - Data Type : String
  - - Description : Link to GeoJSON detail feed from a GeoJSON summary feed.
  NOTE: When searching and using geojson with callback, no callback is included in the detail url. :


** dmin **
  - Data Type : Decimal
  - Typical Values : [0.4, 7.1]
  - Description :Horizontal distance from the epicenter to the nearest station (in degrees). 1 degree is approximately 111.2 kilometers. In general, the smaller this number, the more reliable is the calculated depth of the earthquake.



** felt **
  - Data Type : Integer
  - Typical Values : [44, 843]
  - Description :The total number of felt reports submitted to the DYFI? system.



** gap **
  - Data Type : Decimal
  - Typical Values : [0.0, 180.0]
  - Description :The largest azimuthal gap between azimuthally adjacent stations (in degrees). In general, the smaller this number, the more reliable is the calculated horizontal position of the earthquake. Earthquake locations in which the azimuthal gap exceeds 180 degrees typically have large location and depth uncertainties.


** ids **
  - Data Type : String
  - Typical Values : ",ci15296281,us2013mqbd,at00mji9pf,"
  - Description :A comma-separated list of event ids that are associated to an event.


** mag **
  - Data Type : Decimal
  - Typical Values : [-1.0, 10.0]
  - Description :The magnitude for the event. See also magType.
  - Additional Information : The magnitude reported is that which the U.S. Geological Survey considers official for this earthquake, and was the best available estimate of the earthquake’s size, at the time that this page was created. Other magnitudes associated with web pages linked from here are those determined at various times following the earthquake with different types of seismic data. Although they are legitimate estimates of magnitude, the U.S. Geological Survey does not consider them to be the preferred "official" magnitude for the event. Earthquake magnitude is a measure of the size of an earthquake at its source. It is a logarithmic measure. At the same distance from the earthquake, the amplitude of the seismic waves from which the magnitude is determined are approximately 10 times as large during a magnitude 5 earthquake as during a magnitude 4 earthquake. The total amount of energy released by the earthquake usually goes up by a larger factor: for many commonly used magnitude types, the total energy of an average earthquake goes up by a factor of approximately 32 for each unit increase in magnitude. There are various ways that magnitude may be calculated from seismograms. Different methods are effective for different sizes of earthquakes and different distances between the earthquake source and the recording station. The various magnitude types are generally defined so as to yield magnitude values that agree to within a few-tenths of a magnitude-unit for earthquakes in a middle range of recorded-earthquake sizes, but the various magnitude-types may have values that differ by more than a magnitude-unit for very large and very small earthquakes as well as for some specific classes of seismic source. This is because earthquakes are commonly complex events that release energy over a wide range of frequencies and at varying amounts as the faulting or rupture process occurs. The various types of magnitude measure different aspects of the seismic radiation (e.g., low-frequency energy vs. high-frequency energy). The relationship among values of different magnitude types that are assigned to a particular seismic event may enable the seismologist to better understand the processes at the focus of the seismic event. The various magnitude-types are not all available at the same time for a particular earthquake. Preliminary magnitudes based on incomplete but rapidly-available data are sometimes estimated and reported. For example, the Tsunami Warning Centers will calculate a preliminary magnitude and location for an event as soon as sufficient data are available to make an estimate. In this case, time is of the essence in order to broadcast a warning if tsunami waves are likely to be generated by the event. Such preliminary magnitudes are superseded by improved estimates of magnitude as more data become available. For large earthquakes of the present era, the magnitude that is ultimately selected as the preferred magnitude for reporting to the public is commonly a moment magnitude that is based on the scalar seismic-moment of an earthquake determined by calculation of the seismic moment-tensor that best accounts for the character of the seismic waves generated by the earthquake. The scalar seismic-moment, a parameter of the seismic moment-tensor, can also be estimated via the multiplicative product rigidity of faulted rock x area of fault rupture x average fault displacement during the earthquake.


** magType **
  - Data Type : String
  - Typical Values : “Md”, “Ml”, “Ms”, “Mw”, “Me”, “Mi”, “Mb”, “MLg”
  - Description :The method or algorithm used to calculate the preferred magnitude for the event
    
    
** mmi **
  - Data Type : Decimal
  - Typical Values : [0.0, 10.0]
  - Description :The maximum estimated instrumental intensity for the event. Computed by ShakeMap. While typically reported as a roman numeral, for the purposes of this API, intensity is expected as the decimal equivalent of the roman numeral. Learn more about magnitude vs. intensity.

** nst **
  - Data Type : Integer
  - Description : The total number of seismic stations used to determine earthquake location.
  - Additional Information :Number of seismic stations which reported P- and S-arrival times for this earthquake. This number  : ay be larger than the Number of Phases Used if arrival times are rejected because the distance to a seismic station exceeds the maximum allowable distance or because the arrival-time observation is inconsistent with the solution.



** place **
  - Data Type : String
  - - Description : Textual description of named geographic region near to the event. This may be a city name, or a Flinn-Engdahl Region name.
  - Additional Information : We use a GeoNames dataset to reference populated places that are in close proximity to a seismic event. GeoNames has compiled a list of cities in the United States where the population is 1,000 or greater (cities1000.txt). This is the primary list that we use when selecting nearby places. In order to provide the public with a better understanding for the location of an event we try to list a variety of places in our nearby places list. This includes the closest known populated place in relation to the seismic event (which based on our dataset will have a population of 1,000 or greater). We also include the next 3 closest places that have a population of 10,000 or greater, and finally make sure to include the closest capital city to the seismic event. The reference point for the descriptive locations is usually either the City Hall of the town (or prominent intersection in the middle of town if there is no City Hall), but please refer to the GeoNames website for the most accurate information on their data. If there is no nearby city within 300 kilometers (or if the nearby cities database is unavailable for some reason), the Flinn-Engdahl (F-E) seismic and geographical regionalization scheme is used. The boundaries of these regions are defined at one-degree intervals and therefore differ from irregular political boundaries. For example, F-E region 545 (Northern Italy) also includes small parts of France, Switzerland, Austria and Slovenia and F-E region 493 (Chesapeake Bay Region) includes all of the State of Delaware, plus parts of the District of Columbia, Maryland, New Jersey, Pennsylvania and Virginia. Beginning with January 2000, the 1995 revision to the F-E code has been used in the QED and PDE listings. As an agency of the U.S. Government, we are expected to use the names and spellings approved by the U.S. Board on Geographic Names. Any requests to approve additional names should be made to the U.S. Board on Geographic Names.



** rms **
  - Data Type : Decimal
  - Typical Values : [0.13,1.39]
  - Description :The root-mean-square (RMS) travel time residual, in sec, using all weights. This parameter provides a measure of the fit of the observed arrival times to the predicted arrival times for this location. Smaller numbers reflect a better fit of the data. The value is dependent on the accuracy of the velocity model used to compute the earthquake location, the quality weights assigned to the arrival time data, and the procedure used to locate the earthquake.



** sig **
  - Data Type : Integer
  - Typical Values : [0, 1000]
  - Description :A number describing how significant the event is. Larger numbers indicate a more significant event. This value is determined on a number of factors, including: magnitude, maximum MMI, felt reports, and estimated impact.



** sources **
  - Data Type : String
  - Typical Values : ",us,nc,ci,"
  - Description :A comma-separated list of network contributors.



** status **
  - Data Type : String
  - Typical Values : “automatic”, “reviewed”, “deleted”
  - Description :Indicates whether the event has been reviewed by a human.
  - Additional Information : Status is either automatic or reviewed. Automatic events are directly posted by automatic processing systems and have not been verified or altered by a human. Reviewed events have been looked at by a human. The level of review can range from a quick validity check to a careful reanalysis of the event.



** time **
  - Data Type : Long Integer
  - Description : Time when the event occurred. Times are reported in milliseconds since the epoch ( 1970-01-01T00:00:00.000Z), and do not include leap seconds. In certain output formats, the date is formatted for readability.
  - Additional Information : We indicate the date and time when the earthquake initiates rupture, which is known as the "origin" time. Note that large earthquakes can continue rupturing for many 10's of seconds. We provide time in UTC (Coordinated Universal Time). Seismologists use UTC to avoid confusion caused by local time zones and daylight savings time. On the individual event pages, times are also provided for the time at the epicenter, and your local time based on the time your computer is set.



** tsunami **
  - Data Type : Integer
  - Description : This flag is set to "1" for large events in oceanic regions and "0" otherwise. The existence or value of this flag does not indicate if a tsunami actually did or will exist. If the flag value is "1", the event will include a link to the NOAA Tsunami website for tsunami information. The USGS is not responsible for Tsunami warning; we are simply providing a link to the authoritative NOAA source. :See http://www.tsunami.gov/ for all current tsunami alert statuses.



** type **
  - Data Type : String
  - Typical Values : “earthquake”, “quarry”
  - Description :Type of seismic event.



** types **
  - Data Type : String
  - Typical Values : “,cap,dyfi,general-link,origin,p-wave-travel-times,phase-data,”
  - Description :A comma-separated list of product types associated to this event.


** tz **
  - Data Type : Integer
  - Typical Values : [-1200, +1200]
  - Description :Timezone offset from UTC in minutes at the event epicenter.


** url **
  - Data Type : String
  - - Description : Link to USGS Event Page for event


In [9]:
#Connecting to MySQL Local Server

user = os.getenv('MYSQL_USER')
passwd = os.getenv('MYSQL_PASSWORD')

try:
    conn = msql.connect(host='localhost',database= 'event_2017', user=user, password=passwd)
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database ();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
        cursor.execute('DROP TABLE IF EXISTS earthquake_date;')
        print('Creating table....')
# in the below line please pass the create table statement which you want #to create
        cursor.execute("CREATE TABLE IF NOT EXISTS earthquake_data(event_id CHAR(2),place DECIMAL,dmin DECIMAL,mag DECIMAL,alert VARCHAR(10),gap DECIMAL,sig INTEGER,mag_type CHAR(2),tsunami BINARY,product_type VARCHAR(10),earthquake_type VARCHAR(10),detail VARCHAR(255),occurence_time DATETIME,occurence_hour INT,updated TIMESTAMP,tz INT,url VARCHAR(255),cdi DECIMAL,felt INT,review_status VARCHAR(10),net CHAR(2),rms DECIMAL,mmi DECIMAL,sources VARCHAR(10),nst INT,title VARCHAR(255),mag_bins VARCHAR(3),event_code VARCHAR(15))")
        print("Table is created....")
        #loop through the data frame
        for i,row in data.iterrows():
            # %S means string values 
            sql = "INSERT INTO event_2017.earthquake_data VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            cursor.execute(sql, tuple(row))
            print("Record inserted")
            conn.commit()
except Error as e:
            print("Error while connecting to MySQL", e)

You're connected to database:  ('event_2017')
Creating table....
Table is created....
Record inserted


In [10]:
#Provide query/analysis to give biggest earthquake of 2017

# Executing SQL query
query1 = "SELECT * FROM event_2017.earthquake_data ea WHERE (SELECT MAX(mag) FROM earthquake_data) "
cursor.execute(query1)
# Fetching the record
result = cursor.fetchall()
print(result)


,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,occurence_types,nst,dmin,rms,gap,magType,product_type,title,occurence(hour),mag_bins
97426,8.2,"near the coast of Chiapas, Mexico",2017-09-08 00:49:19,1634735098105,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,2643.0,9.1,7.398,...,",dyfi,finite-fault,general-link,general-text,g...",NaN,0.944,1.22,22.0,mww,earthquake,"M 8.2 - near the coast of Chiapas, Mexico",00,6+


In [11]:
# Provide query/analysis to give most probable hour of the day for the earthquakes bucketed by the range of magnitude (0-1,1-2,2-3,3-4,4-5,5-6,>6   For border values in the bucket, include them in the bucket where the value is a lower limit so for 1 include it in 1-2 bucket)

query2 = "SELECT occurence(hour), mag_bins, Count(place) from event_2017.earthquake_data group by occurence(hour), mag_bins"
cursor.execute(query2)
# Fetch all the records
result = cursor.fetchall()
for i in result:
    print(i)

,occurence(hour),mag_bins,place
0,00,0-1,2027
1,00,1-2,2121
2,00,2-3,639
3,00,3-4,152
4,00,4-5,411
...,...,...,...
163,23,2-3,667
164,23,3-4,145
165,23,4-5,379
166,23,5-6,29
